In [1]:
def get_random_intervals(min_v=0, max_v=1, min_step=0.1, round_v=1):
    import random
    generate = True
    while(generate):
        left_edge = (round(random.uniform(min_v, max_v),round_v))
        if (left_edge+min_step)<=1:
            right_edge = round(random.uniform(left_edge+min_step, max_v),round_v)
            if ((left_edge==0) and (right_edge==1)) == False:
                generate = False
    return left_edge, right_edge

# Read data

In [2]:
def get_random_pattern(df_input, attributes, k, ratio= 0.3, metric = "d_fpr", ranges_attributes = None, min_samples_size = 100, round_v = 3, n_total_tests=100):
    # We pass the ranges to get_random_intervals
    import random
    pattern_not_found = True
    n_tests = 0
    while(pattern_not_found and n_tests<n_total_tests):
        bias_pattern = {}
        sel_attributes = random.sample(attributes, k=k)
        sel_df = df_input.copy()
        for attribute in sel_attributes:
            not_enough_size = True
            min_input_step = min_samples_size/len(df_input)
            n_iter = 1
            while(not_enough_size and n_iter<=5):
                left_edge, right_edge = get_random_intervals(min_step=min_input_step*n_iter, round_v=round_v)

                support_count_interval = len( sel_df.loc[(sel_df[attribute]>=left_edge)&(sel_df[attribute]<=right_edge)] )
                if support_count_interval>min_samples_size:
                    sel_df = sel_df.loc[(sel_df[attribute]>=left_edge)&(sel_df[attribute]<=right_edge)]
                    not_enough_size = False
                else:
                    n_iter+=1
            if not_enough_size:
                break
            else: 
                bias_pattern[attribute] = [[('>=', left_edge), ('<=', right_edge)]]
        if len(bias_pattern)==k:
            pattern_not_found=False
        else:
            n_tests+=1
    if n_tests>n_total_tests:
        raise ValueError()
    if (k!=len(sel_attributes)) or (k!=len(bias_pattern)):
        raise ValueError()

    return bias_pattern, len(sel_df)

In [3]:
import pandas as pd
import os

dataset_name = "artificial_data_10_10k"
dataset_dir = os.path.join(os.path.curdir, "datasets")

from import_datasets import is_dataset_available
if is_dataset_available(f"{dataset_name}.csv", inputDir=dataset_dir):
    print("From stored")
    df_artificial = pd.read_csv(os.path.join(dataset_dir, f"{dataset_name}.csv"))
else:
    from import_datasets import create_artificial_dataset

    df_artificial = create_artificial_dataset(round_v = 3)
    df_artificial.to_csv(os.path.join(dataset_dir, f"{dataset_name}.csv"), index=False)

From stored


In [4]:
df_artificial.head(3)

,a,b,c,d,e,f,g,h,i,j,class
0,0.076,0.780,0.438,0.723,0.978,0.538,0.501,0.072,0.268,0.500,1
1,0.679,0.804,0.381,0.066,0.288,0.910,0.213,0.452,0.931,0.025,0
2,0.601,0.950,0.230,0.548,0.909,0.133,0.523,0.750,0.669,0.468,0


# Generate biases

In [5]:
attributes = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]
injected_bias_list = []
ratio = 0.5
metric = "d_error"
max_len_pattern = 5
n_times = 10
for k in range(2, max_len_pattern+1):
    for i in range(0, n_times):
        # Try 10 time to identify a pattern with enough instances
        bias_pattern, support = get_random_pattern(df_artificial, attributes, k, round_v = 3, n_total_tests = 100)
        if bias_pattern is not None:
            found = True
        if found:
            injected_bias_list.append((bias_pattern, ratio, metric, support))
        else:
            raise ValueError(k)
len(injected_bias_list)

40

In [6]:
def save_pickle(data, filename):
    import pickle

    with open(
        filename,
        "wb",
    ) as handle:
        pickle.dump(
            data,
            handle,
            protocol=pickle.HIGHEST_PROTOCOL,
        )

In [38]:
#save_list_bias = False
# if save_list_bias:
#    save_pickle(injected_bias_list, os.path.join(dataset_dir, "injected_bias_list.pickle"))

In [8]:
import pickle

with open(os.path.join(dataset_dir, "injected_bias_list.pickle"), "rb") as fp:
    injected_bias = pickle.load(fp)

In [33]:
len(injected_bias)

40